In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
from sklearn import preprocessing
from sklearn import metrics
import seaborn as sns
import random

from os import listdir
from os.path import isfile, join

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.calibration import CalibratedClassifierCV

import csv
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings("ignore")

import yaml
from sklearn.model_selection import LeavePGroupsOut

from time import time
from pypif.pif import dumps
import json 

import saxskit
from saxskit.saxskit.saxs_models import get_data_from_Citrination, train_classifiers

from citrination_client import CitrinationClient

In [2]:
with open("citrination_api_key_ssrl.txt", "r") as g: 
    api_key = g.readline()

a_key = api_key.strip()

client = CitrinationClient(site='https://slac.citrination.com',api_key=a_key ) 

In [3]:
t0 = time()
data = get_data_from_Citrination(client, [1,15,16])
print("It took ", time()-t0)

It took  136.26382303237915


In [4]:
data.head()

,experiment_id,Imax_over_Imean,Imax_sharpness,I_fluctuation,logI_fluctuation,logI_max_over_std,r_fftIcentroid,r_fftImax,q_Icentroid,q_logIcentroid,...,guinier_porod,spherical_normal,diffraction_peaks,G_gp,rg_gp,D_gp,I0_sphere,r0_sphere,sigma_sphere,I0_floor
331,R2,8.24115,1.03932,0.00449226,4.9141,2.21375,0.0969313,0.00185529,0.123778,1.41189,...,1,1,1,None,None,None,None,None,None,0
254,R1,18.6027,1.03299,0.00109437,3.55141,2.78792,0.1076,0.00185529,0.0651242,-0.745962,...,0,1,0,None,None,None,1696.73,33.624,0.05731,0.0838085
506,R3,33.3458,1.69052,0.00233193,73.5187,3.29077,0.064788,0.00185529,0.0675967,0.0299875,...,1,1,1,None,None,None,None,None,None,0
1762,Reaction_D,13.3875,1.02239,0.00138548,4.88402,3.03094,0.106413,0.00238663,0.0721248,0.0903203,...,0,1,0,None,None,None,2970.96,27.7521,0.0858243,0.241332
905,R6,10.3815,1.00617,0.00148256,3.58087,2.6374,0.0999802,0.00185529,0.0884666,-0.426634,...,1,1,0,0.86826,3.57573,None,127.22,23.2869,0.0784964,0.0253997


In [5]:
t0 = time()
train_classifiers(data,  hyper_parameters_search = True)
print("It took ", time()-t0)

It took  203.27568197250366


It accuracy.txt we have: {'unidentified': 0.98659847035019033, 'spherical_normal': 0.98536187966028632, 'guinier_porod': 0.8234492904892422, 'diffraction_peaks': 0.98027763159461145}

In [6]:
df = pd.read_csv('2016_saxs_data/R1/R1_1stcool_1_0001_dz_bgsub.csv')
q_i = np.array(df)

from saxskit.saxskit.saxs_classify import SaxsClassifier
from saxskit.saxskit.saxs_math import profile_spectrum

m = SaxsClassifier()

features = profile_spectrum(q_i)
result = m.run_classifier(features)

dict_keys(['diffraction_peaks', 'guinier_porod', 'spherical_normal', 'unidentified'])


In [7]:
result

OrderedDict([('unidentified', (0, 0.99783153025466897)),
             ('guinier_porod', (array([0]), 0.788913341633809)),
             ('spherical_normal', (array([1]), 0.99702492437129719)),
             ('diffraction_peaks', (array([0]), 0.98252653846052462))])